# 24장 트랜스포머 직접 만들어 보기## 트랜스포머란?트랜스포머는 2017년 "Attention Is All You Need" 논문에서 소개된 혁신적인 딥러닝 아키텍처입니다. RNN이나 CNN 없이 어텐션 메커니즘만으로 시퀀스 데이터를 처리하는 방식을 제안했습니다. 이 모델은 BERT, GPT 등 현대 언어 모델의 기반이 되었습니다.### 트랜스포머의 주요 구성 요소- **셀프 어텐션(Self-Attention)**: 시퀀스 내 모든 토큰 간의 관계를 학습- **멀티헤드 어텐션(Multi-Head Attention)**: 여러 관점에서 어텐션을 계산- **포지셔널 인코딩(Positional Encoding)**: 순서 정보를 모델에 주입- **피드 포워드 네트워크(Feed-Forward Network)**: 비선형 변환 수행- **레이어 정규화(Layer Normalization)**: 학습 안정화- **잔차 연결(Residual Connection)**: 그래디언트 소실 방지## 트랜스포머 인코더를 활용한 긍정 부정 예측트랜스포머 인코더는 입력 시퀀스를 인코딩하여 문맥을 이해하는 역할을 합니다. 이 예제에서는 간단한 감성 분석(긍정/부정 분류)을 통해 트랜스포머 인코더의 원리를 학습합니다.[<img src="https://raw.githubusercontent.com/taehojo/taehojo.github.io/master/assets/images/linktocolab.png" align="left"/> ](https://colab.research.google.com/github/taehojo/deeplearning_4th/blob/master/colab/ch24-colab.ipynb)

In [ ]:
# 필요한 라이브러리 임포트import tensorflow as tffrom tensorflow.keras.models import Modelfrom tensorflow.keras.layers import Dense, Embedding, GlobalAveragePooling1D, LayerNormalization, Dropout, Add, Inputfrom tensorflow.keras.optimizers import Adamimport pandas as pdimport numpy as npfrom sklearn.model_selection import train_test_split# 깃허브에 준비된 데이터를 가져옵니다.!git clone https://github.com/taehojo/data.git# CSV 파일 로드dataframe = pd.read_csv('./data/sentiment_data.csv')# 데이터와 라벨 추출sentences = dataframe['sentence'].tolist()labels = dataframe['label'].tolist()# 임베딩 벡터 크기와 최대 문장 길이 설정embedding_dim = 128  # 단어 임베딩의 차원max_len = 10  # 처리할 최대 단어 수

### 데이터 전처리 과정

In [ ]:
# 토크나이저 초기화 및 텍스트를 시퀀스로 변환tokenizer = tf.keras.preprocessing.text.Tokenizer()tokenizer.fit_on_texts(sentences)  # 문장을 단어 인덱스로 변환하기 위한 사전 구축sequences = tokenizer.texts_to_sequences(sentences)  # 문장을 숫자 시퀀스로 변환word_index = tokenizer.word_index  # 단어-인덱스 매핑 사전# 패딩을 사용하여 시퀀스 길이를 동일하게 맞춤data = tf.keras.preprocessing.sequence.pad_sequences(sequences, maxlen=max_len, padding='post')# 데이터셋을 훈련 세트와 검증 세트로 분리X_train, X_val, y_train, y_val = train_test_split(data, labels, test_size=0.2, random_state=42)

### 포지셔널 인코딩(Positional Encoding)트랜스포머는 순환 구조가 없어 시퀀스의 순서 정보를 별도로 제공해야 합니다. 이를 위해 포지셔널 인코딩을 사용합니다.

In [ ]:
# 포지셔널 인코딩 함수def get_positional_encoding(max_len, d_model):    """    max_len: 시퀀스 최대 길이    d_model: 임베딩 차원        수식: PE(pos, 2i) = sin(pos/10000^(2i/d_model))         PE(pos, 2i+1) = cos(pos/10000^(2i/d_model))    """    pos_enc = np.zeros((max_len, d_model))    for pos in range(max_len):        for i in range(0, d_model, 2):            pos_enc[pos, i] = np.sin(pos / (10000 ** (2 * i / d_model)))            if i + 1 < d_model:                pos_enc[pos, i + 1] = np.cos(pos / (10000 ** (2 * (i + 1) / d_model)))    return pos_enc# 포지셔널 인코딩 생성positional_encoding = get_positional_encoding(max_len, embedding_dim)

### 멀티헤드 셀프 어텐션 레이어(Multi-Head Self-Attention Layer)셀프 어텐션은 트랜스포머의 핵심 메커니즘으로, 시퀀스 내 각 위치가 다른 모든 위치와 얼마나 관련있는지 계산합니다.

In [ ]:
# 멀티헤드 어텐션 레이어 (마스크 미사용)class MultiHeadSelfAttentionLayer(tf.keras.layers.Layer):    def __init__(self, num_heads, key_dim):        """        num_heads: 어텐션 헤드 수 (서로 다른 관점에서 어텐션 계산)        key_dim: 키 벡터의 차원        """        super(MultiHeadSelfAttentionLayer, self).__init__()        self.mha = tf.keras.layers.MultiHeadAttention(num_heads=num_heads, key_dim=key_dim)        self.norm = LayerNormalization()  # 레이어 정규화    def call(self, x):        # 셀프 어텐션 계산 (쿼리, 키, 값이 모두 동일한 입력)        attn_output = self.mha(query=x, value=x, key=x)        # 잔차 연결(residual connection)과 레이어 정규화        attn_output = self.norm(attn_output + x)        return attn_output

### 모델 구성: 트랜스포머 인코더

In [ ]:
# 모델 설정inputs = Input(shape=(max_len,))# 1. 임베딩 레이어 - 단어 인덱스를 벡터로 변환embedding_layer = Embedding(input_dim=len(word_index) + 1, output_dim=embedding_dim)embedded_sequences = embedding_layer(inputs)# 2. 포지셔널 인코딩 추가 - 위치 정보 주입embedded_sequences_with_positional_encoding = embedded_sequences + positional_encoding# 3. 첫 번째 멀티헤드 어텐션 (마스크 없음)attention_layer = MultiHeadSelfAttentionLayer(num_heads=8, key_dim=embedding_dim)attention_output = attention_layer(embedded_sequences_with_positional_encoding)# 4. 잔차 연결 - 그래디언트 소실 방지attention_output_with_residual = Add()([embedded_sequences_with_positional_encoding, attention_output])# 5. GlobalAveragePooling1D - 시퀀스를 하나의 벡터로 변환pooled_output = GlobalAveragePooling1D()(attention_output_with_residual)# 6. 피드 포워드 네트워크 - 최종 분류dense_layer = Dense(128, activation='relu')(pooled_output)dropout_layer = Dropout(0.5)(dense_layer)  # 과적합 방지output_layer = Dense(1, activation='sigmoid')(dropout_layer)  # 이진 분류(긍정/부정)model = Model(inputs=inputs, outputs=output_layer)model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

### 모델 학습 및 예측

In [ ]:
# 모델 학습model.fit(X_train, np.array(y_train), epochs=10, batch_size=16, validation_data=(X_val, np.array(y_val)))# 새 문장 예측sample_texts = ["I absolutely love this!", "I can't stand this product"]sample_sequences = tokenizer.texts_to_sequences(sample_texts)sample_data = tf.keras.preprocessing.sequence.pad_sequences(sample_sequences, maxlen=max_len, padding='post')predictions = model.predict(sample_data)for i, text in enumerate(sample_texts):    print(f"Text: {text}")    print(f"Prediction: {'Positive' if predictions[i] > 0.5 else 'Negative'}")

**출력 결과 해석:**학습이 진행됨에 따라 훈련 및 검증 정확도가 빠르게 증가하는 것을 확인할 수 있습니다. 10번의 에포크 후에는 검증 세트에서 99.75%의 높은 정확도를 달성했습니다. 샘플 예측 결과도 직관적으로 맞게 분류되었습니다.## 전체 트랜스포머 구현하기이제 마스킹(masking)을 포함한 전체 트랜스포머 모델을 구현해 보겠습니다. 마스킹은 디코더에서 미래 정보를 보지 못하게 하는 메커니즘입니다.

In [ ]:
# 멀티헤드 어텐션 레이어(마스크 지원)class MultiHeadSelfAttentionLayer(tf.keras.layers.Layer):    def __init__(self, num_heads, key_dim, masked=False):        """        num_heads: 어텐션 헤드 수        key_dim: 키 벡터의 차원        masked: 마스크 사용 여부 (미래 토큰을 보지 못하게 함)        """        super(MultiHeadSelfAttentionLayer, self).__init__()        self.mha = tf.keras.layers.MultiHeadAttention(num_heads=num_heads, key_dim=key_dim)        self.norm = LayerNormalization()        self.masked = masked    def call(self, x):        if self.masked:            # 마스크 생성 (하삼각 행렬 형태)            batch_size = tf.shape(x)[0]            seq_len = tf.shape(x)[1]            mask = tf.linalg.band_part(tf.ones((seq_len, seq_len)), -1, 0)  # 하삼각 행렬            mask = tf.reshape(mask, (1, 1, seq_len, seq_len))            mask = tf.tile(mask, [batch_size, 1, 1, 1])  # 배치 크기에 맞게 복제            mask = (1-mask) * -1e9  # 마스킹할 위치를 매우 작은 값(-∞)으로 설정            attn_output = self.mha(query=x, value=x, key=x, attention_mask=mask)        else:            attn_output = self.mha(query=x, value=x, key=x)        attn_output = self.norm(attn_output + x)  # 잔차 연결 및 레이어 정규화        return attn_output

### 전체 트랜스포머 모델 구성

In [ ]:
# 모델 설정inputs = Input(shape=(max_len,))# 1. 임베딩 레이어embedding_layer = Embedding(input_dim=len(word_index) + 1, output_dim=embedding_dim)embedded_sequences = embedding_layer(inputs)# 2. 포지셔널 인코딩 추가embedded_sequences_with_positional_encoding = embedded_sequences + positional_encoding# 3. 첫 번째 멀티헤드 어텐션 (마스크 없음) - 인코더 부분attention_layer = MultiHeadSelfAttentionLayer(num_heads=8, key_dim=embedding_dim)attention_output = attention_layer(embedded_sequences_with_positional_encoding)# 4. 잔차 연결attention_output_with_residual = Add()([embedded_sequences_with_positional_encoding, attention_output])# 5. 마스크드 멀티헤드 어텐션 (마스크 있음) - 디코더 부분masked_attention_layer = MultiHeadSelfAttentionLayer(num_heads=8, key_dim=embedding_dim, masked=True)masked_attention_output = masked_attention_layer(attention_output_with_residual)# 6. 잔차 연결masked_attention_output_with_residual = Add()([attention_output_with_residual, masked_attention_output])# 7. GlobalAveragePooling1Dpooled_output = GlobalAveragePooling1D()(masked_attention_output_with_residual)# 8. 피드 포워드 네트워크dense_layer = Dense(128, activation='relu')(pooled_output)dropout_layer = Dropout(0.5)(dense_layer)output_layer = Dense(1, activation='sigmoid')(dropout_layer)model = Model(inputs=inputs, outputs=output_layer)model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

### 마스킹의 의미와 시각화마스킹은 디코더에서 자기회귀적(autoregressive) 특성을 구현하기 위해 중요합니다. 각 위치는 자신보다 앞선 위치의 정보만 볼 수 있습니다.

In [ ]:
# 마스크 행렬 시각화 예제seq_len_example = 4batch_size_example = 2mask_example = tf.linalg.band_part(tf.ones((seq_len_example, seq_len_example)), -1, 0)mask_example = tf.reshape(mask_example, (1, 1, seq_len_example, seq_len_example))mask_example = tf.tile(mask_example, [batch_size_example, 1, 1, 1])print("\n예시 마스크 행렬 :")print(mask_example.numpy()[0, 0, :, :])# 하삼각 행렬 형태:# 1 0 0 0# 1 1 0 0# 1 1 1 0# 1 1 1 1mask_example = (1-mask_example) * -1e9print("\n-∞로 치환한 마스크 행렬:")print(mask_example.numpy()[0, 0, :, :])# 0 -1e9 -1e9 -1e9# 0    0 -1e9 -1e9# 0    0    0 -1e9# 0    0    0    0

## 트랜스포머 모델의 주요 특징 요약1. **병렬 처리**: RNN과 달리 모든 위치를 병렬로 처리하여 학습 속도가 빠름2. **장거리 의존성 포착**: 어텐션 메커니즘으로 문장 내 멀리 떨어진 단어 간의 관계도 효과적으로 학습3. **확장성**: 계층을 추가하여 더 복잡한 패턴 학습 가능4. **적응성**: 다양한 NLP 태스크에 적용 가능## 실습 과제1. 최대 문장 길이(max_len)를 다양하게 변경하며 성능 변화 관찰하기2. 어텐션 헤드 수(num_heads)를 조정하여 결과 비교하기3. 다른 감성 분석 데이터셋에 적용해보기4. 트랜스포머 레이어를 더 추가하여 모델의 깊이가 성능에 미치는 영향 확인하기## 참고 자료- "Attention Is All You Need" 논문 (Vaswani et al., 2017)- [트랜스포머 모델 시각화](http://jalammar.github.io/illustrated-transformer/)- [텐서플로우 공식 튜토리얼: 트랜스포머](https://www.tensorflow.org/tutorials/text/transformer)